<h1><b>Ministry Classification Model</b></h1>

<h3>We will be only considering the first 550 character of the (title + content) for our prediction, Due to the computational limitation</h3>

<h2>
<h2>TODO</h2>
<ul>
<li>Padding length is to be set, According to the result wanted, Currently padding length is seted to the `550`.</li>    
</ul>
</h2>

<h1>Process 1: Importing the Libraries</h1>

In [125]:
# For basic dataframe and mathematical operations
import numpy as np
import pandas as pd

# For Deep Learning Model
import tensorflow as tf
from tensorflow import keras
import os

# For natural language processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# For utility
import re
# import lgging
import time
import pickle
import itertools


<h2>Processs 2: Code for configuring multiple GPUs</h2>

In [126]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7110176208523214213
xla_global_id: -1
]


<h1>Process 3: Making press release dataset ready for giving it as input to classification model</h1>

<h2>Managining new 61th i.e. no ministry labeled datatset for classification model</h2>

In [127]:
df = pd.read_csv("/kaggle/input/indian-news-dataset-with-ministry-labels/combined_dataset.csv", encoding='unicode_escape')

<h2>Managing press release dataset</h2>

<h2>List of the ministry and it's specific index number</h2>
<ul>
<li> 0 Ministry of Railways </li>
<li> 1 Ministry of Rural Development </li>
<li> 2 Ministry of Steel </li>
<li> 3 Ministry of Science & Technology </li>
<li> 4 Ministry of Information & Broadcasting </li>
<li> 5 Ministry of Food Processing Industries </li>
<li> 6 Ministry of Health and Family Welfare </li>
<li> 7 Ministry of Human Resource Development </li>
<li> 8 Ministry of Agriculture </li>
<li> 9 Ministry of Environment and Forests </li>
<li> 10 Ministry of Chemicals and Fertilizers </li>
<li> 11 Ministry of Water Resources </li>
<li> 12 Ministry of Defence </li>
<li> 13 Ministry of Petroleum & Natural Gas </li>
<li> 14 President's Secretariat </li>
<li> 15 Ministry of Micro,Small & Medium Enterprises </li>
<li> 16 Ministry of Mines </li>
<li> 17 Ministry of Tourism </li>
<li> 18 Ministry of Housing & Urban Affairs </li>
<li> 19 Ministry of Coal </li>
<li> 20 Prime Minister's Office </li>
<li> 21 Ministry of Textiles </li>
<li> 22 Ministry of Commerce & Industry </li>
<li> 23 Ministry of External Affairs </li>
<li> 24 Ministry of Social Justice & Empowerment </li>
<li> 25 Ministry of Power </li>
<li> 26 Ministry of Consumer Affairs, Food & Public Distribution </li>
<li> 27 Ministry of Heavy Industries & Public Enterprises </li>
<li> 28 Ministry of Communications </li>
<li> 29 Ministry of Shipping </li>
<li> 30 Ministry of Finance </li>
<li> 31 Ministry of Tribal Affairs </li>
<li> 32 Ministry of Statistics & Programme Implementation </li>
<li> 33 Ministry of Labour & Employment </li>
<li> 34 Ministry of Law & Justice </li>
<li> 35 Vice President's Secretariat </li>
<li> 36 Ministry of Civil Aviation </li>
<li> 37 Ministry for Development of North-East Region </li>
<li> 38 UPSC </li>
<li> 39 Ministry of Agro & Rural Industries </li>
<li> 40 Ministry of Home Affairs </li>
<li> 41 Ministry of Youth Affairs and Sports </li>
<li> 42 Special Service and Features </li>
<li> 43 Ministry of New and Renewable Energy </li>
<li> 44 Ministry of Parliamentary Affairs </li>
<li> 45 Planning Commission </li>
<li> 46 Ministry of Personnel, Public Grievances & Pensions </li>
<li> 47 Election Commission </li>
<li> 48 Department of Space </li>
<li> 49 Ministry of Disinvestment </li>
<li> 50 Department of Ocean Development </li>
<li> 51 Ministry of Overseas Indian Affairs </li>
<li> 52 Ministry of Housing and Urban Poverty Alleviation </li>
<li> 53 Ministry of Culture </li>
<li> 54 Ministry of Company Affairs </li>
<li> 55 Ministry of Panchayati Raj </li>
<li> 56 Cabinet Committee on Economic Affairs (CCEA) </li>
<li> 57 Cabinet </li>
<li> 58 Department of Atomic Energy </li>
<li> 59 Cabinet Committee Decisions </li>
<li>60 No Ministry</li>
</ul>

<b>Mapping the ministry to the it's index number starting from the 1 and `0` is made reserved for the `Not Ministry`, If any news doesn't belongs to any ministry of our current ministry dataset, So total labels are `59+1(no ministry)`</b>

<h3>Making the cocatination of the pr_title and pr_content in the new field named content</h3>

In [128]:
tmp =  df.labels.isnull()
for idx, val in enumerate(tmp):
    if val == True:
        print(df['news_num'][idx], df['labels'][idx], df['content'][idx])
        print('\n')

In [129]:
for i in range(len(df)):       
    if pd.isnull(df.loc[i, 'title']):
        df.loc[i, 'content'] = str(df.loc[i, 'content'])[:550]
    elif pd.isnull(df.loc[i, 'content']):
        df.loc[i, 'content'] = str(df.loc[i, 'title'])[:550]
    else:
        df.loc[i, 'content'] = (str(df.loc[i, 'title']) + " " + str(df.loc[i, 'content']))[:550]
df.drop(columns=['news_num', 'title'], inplace=True)

In [130]:
df.head()

,labels,content
0,0,"Trainer aircraft crashes in Odisha, 2 killed A..."
1,17,Uttarkhand unlock 1.0: Hotel bookings for mini...
2,12,J-K: Four Hizbul militants killed in Shopian e...
3,"54,33","Mumbai offices to reopen today, with curbs Pri..."
4,12,"PDP, NC, PC call for release of all J&K leader..."


<h2>Concatinating press release and 0 label dataset</h2>

In [131]:
# df = pd.concat([df, df4, df5, df6], ignore_index=True)

In [132]:
# df.head()

In [133]:
# df.describe(include='object')

<h3>Shuffling Dataframe</h3>

In [134]:
# df = df.sample(frac = 1)

<h2>Preprocessing and Lemmatizing the text, So that makning the text to the it's base form. So that there would be no repetation allotment of the same keyword in the different state</h2>

In [135]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [136]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [137]:
def preprocess(text, stem=False):
    # Removing link, user, special characters
    test = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [138]:
for i in range(len(df)):
    df.loc[i, 'content'] = preprocess(df.loc[i, 'content'], True)

<h1>Doing OneHot encoding of the labels</h2>

In [139]:
df['labels'][458]

'6,7'

In [140]:
# def convert_to_int_list(x):
#     try:
#         return [int(value) for value in x.split(',') if value]
#     except ValueError:
#         return []


# tmp_label_list = df['labels'].apply(convert_to_int_list)

In [141]:
# tmp_label_list[3]

<h2>Making the custom one hot labeling function as per the requriement</h2>

In [142]:
# def make_one_hot_encoding(num_ministries, tmp_label_list):
#     outer_list = []

#     for i in range(len(tmp_label_list)):
#         inner_list = [0]*num_ministries
#         for j in tmp_label_list[i]:
#             inner_list[j] = 1
#         outer_list.append(inner_list)
#     return outer_list

In [143]:
# num_classes = 61

# # onehot_labels = make_one_hot_encoding(61, tmp_label_list)
# onehot_labels = []
# for label_list in tmp_label_list:
#     categorical = tf.keras.utils.to_categorical(label_list, num_classes=num_classes)
#     if len(categorical) >1:
#         i=0
#         for list_ in categorical:
#             if i==0:
#                 ans = np.asarray(list_.tolist())
#             else:
#                 ans = ans + np.asarray(list_.tolist())
#             i+=1
#         onehot_labels.append(ans)
#     else:
#         onehot_labels.append(np.asarray(categorical.flatten().tolist()))

# onehot_labels = np.asarray(onehot_labels)

In [170]:
def convert_to_multi_one_hot(df, label_col, num_classes):
    
    # Split labels by comma  
    labels = df[label_col].apply(lambda x: x.split(",")).values
    # Initialize one-hot encoding array  
    one_hot = np.zeros((len(labels), num_classes))
    for i, label_vals in enumerate(labels):
        # Clean individual values
        cleaned = [] 
        for l in label_vals:
            l = l.strip().rstrip(".") # Strip whitespace and periods
            try:
                index = int(l) # Attempt integer conversion
                cleaned.append(index) 
            except ValueError: 
                print(f"Skipping invalid label value {l}")
                pass # Skip invalid values
                
            # Set encoding from cleaned values
            indices = [int(l) for l in cleaned]  
            one_hot[i, indices] = 1 
    return one_hot

onehot_labels = convert_to_multi_one_hot(df, "labels", 61)

Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 
Skipping invalid label value 41`


In [172]:
print(onehot_labels[0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [173]:
onehot_labels[3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

<h2>Tokenising Text</h2>

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df.content)

vocab_size = len(tokenizer.word_index) + 1
print("Total Words", vocab_size)

<b>For saving the tokenizer</b>

In [ ]:
# ALREADY COMMENTED FROM BEGINING, THIS PART IS NOT COMMENTED DURING THE DATASET CHANGE

# import json

# # Serialize the tokenizer's configuration to JSON
# tokenizer_config = tokenizer.to_json()

# # Save the tokenizer configuration to a file
# with open('tokenizer_config.json', 'w', encoding='utf-8') as json_file:
#     json.dump(tokenizer_config, json_file, ensure_ascii=False, indent=4)

# # Optionally, save the tokenizer weights to a separate file (e.g., for the embeddings layer)
# # tokenizer.word_index contains the vocabulary and word-to-index mapping
# with open('tokenizer_weights.json', 'w', encoding='utf-8') as json_file:
#     json.dump(tokenizer.word_index, json_file, ensure_ascii=False, indent=4)

<b>For converting the text to the sequence of tokens</b>

In [ ]:
tokenized_train_text = tokenizer.texts_to_sequences(df.content)
# tokenized_train_text = same for test

<h2><b>Currently setting the the padding length to be equal to the maxlength of the content, But in the applciacation one length have to be made fixed for this pupose, if cotent is more than that then the content will be croped</b></h2>

<b>OR</b>

<h2>
<b>
    One have to change the input layer dimension of the model, during the inference time
</b>
<h2>
<b>It is remaining to make the test dataset, And by help of the padding make sure in it that the input size is same as that of the input layer, otherwise it will give an error</b>

<b>Finding the max length of the content from the dataset, To make it the height text or sequence length for all of one, So that the input would have the same dimensions by help of the padding</b>

In [ ]:
# max_sequence_len_tmp = max([len(x) for x in df.content])
max_sequence_len_tmp = 550 # As per the EDA report of the dataset by Neel Shah

<h2>TODO: Also check for the pre</h2>

In [ ]:
x_train = keras.preprocessing.sequence.pad_sequences(tokenized_train_text, maxlen=max_sequence_len_tmp, truncating='post',)
# x_test = same for test as of train

In [ ]:
x_train[2]

next, to do label encdcoder, whicg encodes the value betn 0 and 1, but 

In [ ]:
# Embedding dimensions to be changed as per the in which highest accuracy is got
import tensorflow as tf

# Create a MirroredStrategy to use all available GPUs
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, 300, input_length=max_sequence_len_tmp),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(61, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.0010), metrics=['accuracy'])
    model.summary()

In [ ]:
type(x_train[0])

In [ ]:
type(onehot_labels[0]), type(onehot_labels[33000])

In [ ]:
len(onehot_labels)

In [ ]:
x_train = tf.convert_to_tensor(x_train)
type(x_train)

In [ ]:
onehot_labels[0]

In [ ]:
C[3]

In [174]:
# flat_labels = np.array([np.array(x) for x in onehot_labels])

array([array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])                           ,
       array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])                           ,
       array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])                           ,
       ...,
 

In [177]:
onehot_labels = tf.convert_to_tensor(onehot_labels, dtype=tf.float32)
# type(onehot_labels)

In [ ]:
num_epochs = 30

import tensorflow as tf

# Create a MirroredStrategy to use all available GPUs
with strategy.scope():
    callbacks = [ tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
                  tf.keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]
    # with single gpu    
#     history = model.fit(x_train, onehot_labels, batch_size=256, epochs=num_epochs, verbose=1)
    
    # with double gpu
    history = model.fit(x_train, onehot_labels, batch_size=1024, epochs=num_epochs, verbose=1, callbacks=[callbacks])
    
    # history = model.fit(x_train, onehot_labels, epochs=num_epochs, verbose=1)

<b>Plotting the accuracy</b>

In [ ]:
import matplotlib.pyplot as plt

# plot utility
def plot_graphs(history, string):
    plt.plot(history.history[string])
#     plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, string])
    #     plt.legend([string, 'val_'+string])
    plt.show()
    
def plot_val_graphs(history, string):
    plt.plot(history.history['val_'+string])
#     plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel('Validation '+string)
    #     plt.legend([string, 'val_'+string])
    plt.show()
    
# plot the accuracy and results
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

plot_val_graphs(history, "accuracy")
plot_val_graphs(history, "loss")

<h2>Saving the weights</h2>

In [ ]:
KERAS_MODEL = "model.h5"
TOKENIZER_MODEL = "tokenizer.pkl"
    
model.save(KERAS_MODEL)
pickle.dump(tokenizer, open(TOKENIZER_MODEL, "wb"), protocol=0)

<h1>Testing (Error Will Come With GPU, Try Instead Recreating section, without GPU)</h1>

In [ ]:
with strategy.scope():
    tokenized_test_text = tokenizer.texts_to_sequences(["EMPHASIS ON USE OF HINDI IN RAILWAYS The Chairman, Railway Board, Shri R.K.Singh ha"])
    x_train = keras.preprocessing.sequence.pad_sequences(tokenized_test_text, maxlen=max_sequence_len_tmp)
    ans = model.predict([x_train])
    class_labels = ans.argmax(axis=-1)
    print(ans)

In [ ]:
print(class_labels)

<h1>For recreating the model from weights, Without gpu, As error is comming in gpu (NO GPU)</h1>
CancelledError                            Traceback (most recent call last)
Cell In[40], line 4
      2 tokenized_test_text = tokenizer.texts_to_sequences(["EMPHASIS ON USE OF HINDI IN RAILWAYS The Chairman, Railway Board, Shri R.K.Singh ha"])
      3 x_train = keras.preprocessing.sequence.pad_sequences(tokenized_test_text, maxlen=max_sequence_len_tmp)
----> 4 ans = model.predict([x_train])
      5 class_labels = ans.argmax(axis=-1)
      6 print(ans)

File /opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py:70, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     67     filtered_tb = _process_traceback_frames(e.__traceback__)
     68     # To get the full stack trace, call:
     69     # `tf.debugging.disable_traceback_filtering()`
---> 70     raise e.with_traceback(filtered_tb) from None
     71 finally:
     72     del filtered_tb

File /opt/conda/lib/python3.10/site-packages/tensorflow/python/eager/execute.py:52, in quick_execute(op_name, num_outputs, inputs, attrs, ctx, name)
     50 try:
     51   ctx.ensure_initialized()
---> 52   tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
     53                                       inputs, attrs, num_outputs)
     54 except core._NotOkStatusException as e:
     55   if name is not None:

CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node sequential/dense_1/Softmax/_53}}]] [Op:__inference_predict_function_34609]

In [ ]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np

In [ ]:
# As per last execution during training
vocab_size = 40310
max_sequence_len_tmp = 550

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 300, input_length=max_sequence_len_tmp),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(61, activation='softmax')
])  

In [ ]:
model.summary()

<h1>For loading the pretrained model, And making the prediction</h1>

In [ ]:
model.load_weights('/kaggle/input/tmp-data/model.h5')

In [ ]:
def load_tokenizer():
    with open('/kaggle/input/tmp-data/tokenizer.pkl', 'rb') as handle:
        tokenizer = pickle.load(handle)

    print(len(tokenizer.word_index))
    print('loading of tokenizer completed........')

    return tokenizer

tokenizer = load_tokenizer()

In [ ]:
tokenized_test_text = tokenizer.texts_to_sequences(["Supreme court gives notice to car owner regarding tax dues"])
x_train = keras.preprocessing.sequence.pad_sequences(tokenized_test_text, maxlen=max_sequence_len_tmp)
ans = model.predict([x_train])
print(ans)

<h2>Finding Top 3 probable minstries as per the model's output</h2>

In [ ]:
# dict_items([('Not Ministry', 61), ('Ministry of Railways', 1), ('Ministry of Rural Development', 2), ('Ministry of Steel', 3), ('Ministry of Science & Technology', 4), ('Ministry of Information & Broadcasting', 5), ('Ministry of Food Processing Industries', 6), ('Ministry of Health and Family Welfare', 7), ('Ministry of Human Resource Development', 8), ('Ministry of Agriculture', 9), ('Ministry of Environment and Forests', 10), ('Ministry of Chemicals and Fertilizers', 11), ('Ministry of Water Resources', 12), ('Ministry of Defence', 13), ('Ministry of Petroleum & Natural Gas', 14), ("President's Secretariat", 15), ('Ministry of Micro,Small & Medium Enterprises', 16), ('Ministry of Mines', 17), ('Ministry of Tourism', 18), ('Ministry of Housing & Urban Affairs', 19), ('Ministry of Coal', 20), ("Prime Minister's Office", 21), ('Ministry of Textiles', 22), ('Ministry of Commerce & Industry', 23), ('Ministry of External Affairs', 24), ('Ministry of Social Justice & Empowerment', 25), ('Ministry of Power', 26), ('Ministry of Consumer Affairs, Food & Public Distribution', 27), ('Ministry of Heavy Industries & Public Enterprises', 28), ('Ministry of Communications', 29), ('Ministry of Shipping', 30), ('Ministry of Finance', 31), ('Ministry of Tribal Affairs', 32), ('Ministry of Statistics & Programme Implementation', 33), ('Ministry of Labour & Employment', 34), ('Ministry of Law & Justice', 35), ("Vice President's Secretariat", 36), ('Ministry of Civil Aviation', 37), ('Ministry for Development of North-East Region', 38), ('UPSC', 39), ('Ministry of Agro & Rural Industries', 40), ('Ministry of Home Affairs', 41), ('Ministry of Youth Affairs and Sports', 42), ('Special Service and Features', 43), ('Ministry of New and Renewable Energy', 44), ('Ministry of Parliamentary Affairs', 45), ('Planning Commission', 46), ('Ministry of Personnel, Public Grievances & Pensions', 47), ('Election Commission', 48), ('Department of Space', 49), ('Ministry of Disinvestment', 50), ('Department of Ocean Development', 51), ('Ministry of Overseas Indian Affairs', 52), ('Ministry of Housing and Urban Poverty Alleviation', 53), ('Ministry of Culture', 54), ('Ministry of Company Affairs', 55), ('Ministry of Panchayati Raj', 56), ('Cabinet Committee on Economic Affairs (CCEA)', 57), ('Cabinet', 58), ('Department of Atomic Energy', 59), ('Cabinet Committee Decisions', 60)])

In [ ]:
class_labels = ans.argmax(axis=-1)
print(class_labels)
top_3_labels = np.argsort(ans)[-3:]
print(top_3_labels)